In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings(action="ignore")

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/train.csv")
train.head()

In [ ]:
train.shape

In [ ]:
import missingno as msno
msno.matrix(train)

* There are no missing values in the dataset.

In [ ]:
train.columns

In [ ]:
train.target.unique()

In [ ]:
test = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/test.csv")
test.head()

In [ ]:
test.drop(columns="id", inplace=True)

In [ ]:
test.head()

In [ ]:
y = train.target

In [ ]:
y.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y  = le.fit_transform(y)

In [ ]:
train.drop(columns=["id","target"],inplace=True)
y

# Scaling data using StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train = sc.fit_transform(train)
test = sc.transform(test)

# Dimensionality Reduction..
* Applying Principal Component Analysis : Principal Component Analysis, or PCA, is a dimensionality-reduction method that is often used to reduce the dimensionality of large data sets, by transforming a large set of variables into a smaller one that still contains most of the information in the large set.

-- Note: Checkout [this](https://builtin.com/data-science/step-step-explanation-principal-component-analysis) to get better understanding of PCA.


In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(train)

xi = np.arange(1,76,1)
yi = np.cumsum(pca.explained_variance_ratio_)

plt.figure(figsize=(16,8))
plt.plot(xi,yi, marker="p", linestyle="--", color="b")
plt.axhline(y=0.95, color='r',linestyle='-')
plt.text(0.5, 0.85,"95% cut-off Threshold", color="red")
plt.xticks(np.arange(1,80,1))
plt.ylabel("Cummulative Variance %")
plt.xlabel("Number of Components")
plt.grid(axis="x")
plt.show()

In [ ]:
# Reducing components to 70 by observing the graph
pca_new = PCA(n_components=70)
train = pca_new.fit_transform(train)
test = pca_new.transform(test)

In [ ]:
y

# Model Training..🚴

In [ ]:
from lightgbm import LGBMClassifier
classifier = LGBMClassifier(n_estimators=8000,max_depth=6)

In [ ]:
%%time
classifier.fit(train, y)

In [ ]:
y_pred_proba = classifier.predict_proba(test)
y_pred_proba

# Submission...🎯

In [ ]:
output = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv")
output.head()

In [ ]:
submission = pd.DataFrame(y_pred_proba,columns=['Class_1', 'Class_2', 
                                               'Class_3', 'Class_4','Class_5',
                                               'Class_6','Class_7','Class_8','Class_9'])


In [ ]:
test = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/test.csv")

In [ ]:
submission = pd.concat([test.id,output_df],axis = 1)

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission1.csv', index=False)